In [14]:
def int_to_bin(number: int, block_size=8) -> str:
    binary = bin(number)[2:] #convierte a binario el 'number' y regresa un 'string' 
    #'0' * (block_size - len(binary))+binary   #si el tamaño de 'number' es < 32 le agrega 
    # a 'binary' los '0's faltantes hasta 32
    return '0' * (block_size - len(binary)) + binary 


def char_2_num(letter: str) -> int:
    return ord(letter) - ord('a')


def num_2_char(number: int) -> str:
    return chr(ord('a') + number)


def mod(a, b):
    return a % b


def left_circ_shift(binary: str, shift: int) -> str:
    shift = shift % len(binary)
    return binary[shift:] + binary[0: shift]

In [15]:
number = 32896 # En binario esto es: '32896'd = '1000000010000000'b
binary = int_to_bin(number, block_size=16)
print(binary)
#print(char_2_num('g'))
#print(num_2_char(6))
#print(mod(24,11))
#print(left_circ_shift('abcde',2))

1000000010000000


In [16]:
class PBox:
    def __init__(self, clave: dict):
        self.clave = clave
        self.in_degree = len(clave)
        self.out_degree = sum(len(value) if isinstance(value, list) else 1 for value in clave.values())

    def __repr__(self) -> str:
        return 'PBox' + str(self.clave)

    def permutate(self, sequence: list) -> str:
        result = [0] * self.out_degree
        for index, value in enumerate(sequence):
            if (index + 1) in self.clave:
                indices = self.clave.get(index + 1, [])
                indices = indices if isinstance(indices, list) else [indices]
                for i in indices:
                    result[i - 1] = value
        return ''.join(map(str, result))

    def is_invertible(self) -> bool:
        return self.in_degree == self.out_degree

    def invert(self):
        if self.is_invertible():
            result = {}
            for index, mapping in self.clave.items():
                result[mapping] = index
            return PBox(result)

    @staticmethod
    def identity(block_size=64):
        return PBox({index: index for index in range(1, block_size + 1)})

    @staticmethod
    def from_list(permutation: list):
        mapping = {}
        for index, value in enumerate(permutation):
            indices = mapping.get(value, [])
            indices.append(index + 1)
            mapping[value] = indices
        return PBox(mapping)

    @staticmethod
    def des_initial_permutation():
        return PBox.from_list(
            [58, 50, 42, 34, 26, 18, 10, 2,
             60, 52, 44, 36, 28, 20, 12, 4,
             62, 54, 46, 38, 30, 22, 14, 6,
             64, 56, 48, 40, 32, 24, 16, 8,
             57, 49, 41, 33, 25, 17,  9, 1,
             59, 51, 43, 35, 27, 19, 11, 3,
             61, 53, 45, 37, 29, 21, 13, 5,
             63, 55, 47, 39, 31, 23, 15, 7]
        )

        """     
        Implemente en este espacio un método estático 
        para la permutación final. Nombrela como 
        'des_final_permutation()'
        Cf. Fig. 3.7 y 3.9 del libro
        """

    @staticmethod
    def des_final_permutation():
        return PBox.from_list(
            [40, 8, 48, 16, 56, 24, 64, 32,
             39, 7, 47, 15, 55, 23, 63, 31,
             38, 6, 46, 14, 54, 22, 62, 30,
             37, 5, 45, 13, 53, 21, 61, 29,
             36, 4, 44, 12, 52, 20, 60, 28,
             35, 3, 43, 11, 51, 19, 59, 27,
             34, 2, 42, 10, 50, 18, 58, 26,
             33, 1, 41, 9, 49, 17, 57, 25]
        )
    @staticmethod
    def des_single_round_expansion():
        """Esta es la permutación realizada en la mitad derecha del bloque para .. 
        convertir 32 bits --> 48 bits en el DES Mixer"""
        return PBox.from_list(
            [32,  1,  2,  3,  4,  5,
              4,  5,  6,  7,  8,  9,
              8,  9, 10, 11, 12, 13,
             12, 13, 14, 15, 16, 17,
             16, 17, 18, 19, 20, 21,
             20, 21, 22, 23, 24, 25,
             24, 25, 26, 27, 28, 29,
             28, 29, 30, 31, 32,  1 ]
        )

    """
    Implemente um metodo estatico que realice la 
    permutacion al final de la sustitución de cada ronda 
    Nombrela des_single_round_final()
    Cf. Tabla 3.10.
    """

    @staticmethod
    def des_single_round_final():
        return PBox.from_list(
            [16,  7,  20,  21,  29,  12, 28,  17,
             1,  15,  23,  26,  5,  18, 31,  10,
             2,  8,  24,  14,  32,  27, 3,  9,
             19,  13,  30,  6,  22,  11, 4,  25]
        )
   

In [17]:
# Podemos crear un PBox de permutacion que implemente 
# la permutación de la Fig. 3.6
#  
initial_permutation = PBox.des_initial_permutation()
permutationIni = initial_permutation.permutate(int_to_bin(32896, block_size=64))
print('Permutación Inicial:', permutationIni)
print('Long. de Salida:', len(permutationIni))

Permutación Inicial: 0000000000000000000000000000000011000000000000000000000000000000
Long. de Salida: 64


In [18]:
# Podemos crear un PBox de expansión que se expandirá de 32 Bits --> 48 Bits
# Ej. '1,251,434,458'd =  '1001010100101110101111111011010'b 32 bits
# lo expandimos a 48 bits con Fig. 3.11 - Tabla 3.1
# compruebe que el MSB y el LSB asi como algunos bits intermedios 
# están bien colocados
expansion_p_box = PBox.des_single_round_expansion()
permutation = expansion_p_box.permutate(int_to_bin(1251434458, block_size=32))
print('Permutación:', permutation)
print('Long. de Salida:', len(permutation))

Permutación: 001001010101010010101110101011111111111011110100
Long. de Salida: 48


In [19]:
# Podemos crear una P-Box directa
# Explique que esta sucediendo con esta caja de permutacion
straight_p_box = PBox.from_list([4, 1, 3, 5, 2])
p = straight_p_box.permutate('ayuda')
print(p)

dauay


In [20]:
class SBox:
    def __init__(self, table: dict, block_size=4, func=lambda binary: (binary[0] + binary[5], binary[1:5])):
        self.table = table
        self.block_size = block_size
        self.func = func

    def __call__(self, binary: str) -> str:
        a, b = self.func(binary)
        a, b = int(a, base=2), int(b, base=2)
        if (a, b) in self.table:
            return int_to_bin(self.table[(a, b)], block_size=self.block_size)
        else:
            return binary

    @staticmethod
    def des_single_round_substitutions():
        return [SBox.forDESSubstitution(block) for block in range(1, 9)]

    @staticmethod
    def identity():
        return SBox(func=lambda binary: ('0', '0'), table={})

    @staticmethod
    def forDESSubstitution(block):
        if block == 1: return SBox.des_s_box1()
        if block == 2: return SBox.des_s_box2()
        if block == 3: return SBox.des_s_box3()
        if block == 4: return SBox.des_s_box4()
        if block == 5: return SBox.des_s_box5()
        if block == 6: return SBox.des_s_box6()
        if block == 7: return SBox.des_s_box7()
        if block == 8: return SBox.des_s_box8()

    @staticmethod
    def des_confusion(binary: str) -> tuple:
        """"Toma una cadena binaria de 6 bits como entrada y ..
        devuelve una cadena binaria de 4 bits como salida."""
        return binary[0] + binary[5], binary[1: 5]

    @staticmethod
    def from_list(sequence: list):
        mapping = {}
        for row in range(len(sequence)):
            for column in range(len(sequence[0])):
                mapping[(row, column)] = sequence[row][column]
        return SBox(table=mapping)

    @staticmethod
    def des_s_box1():
        return SBox.from_list(
        [[14,  4, 13, 1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9, 0,  7],
         [ 0, 15,  7, 4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5, 3,  8],
         [ 4,  1, 14, 8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10, 5,  0],
         [15, 12,  8, 2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0, 6, 13]]
        )

    """
    Implemente aquí un metodo estatico que codifique 
    2. la caja de substitucion 2 y nombrela 'des_s_box2()'.
    Cf. Tabla 3.3  
    3. la caja de substitucion 3 y nombrela 'des_s_box3()'.
    Cf. Tabla 3.4
    4. la caja de substitucion 4 y nombrela 'des_s_box4()'.
    Cf. Tabla 3.5
    5. la caja de substitucion 5 y nombrela 'des_s_box5()'.
    Cf. Tabla 3.6
    6. la caja de substitucion 6 y nombrela 'des_s_box6()'.
    Cf. Tabla 3.7
    7. la caja de substitucion 7 y nombrela 'des_s_box7()'.
    Cf. Tabla 3.8
    8. la caja de substitucion 8 y nombrela 'des_s_box8()'.
    Cf. Tabla 3.9
    """
    @staticmethod
    def des_s_box2():
        return SBox.from_list(
        [[15,1,8,14,6,11,3,4,9,7,2,13,12,00,5,10],
        [3,13,4,7,15,2,8,14,12,00,1,10,6,9,11,5],
        [00,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
        [13,8,10,1,3,15,4,2,11,6,7,12,00,5,14,9]]
        )
    
    @staticmethod
    def des_s_box3():
        return SBox.from_list([
            [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
            [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
            [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 12]
        ])

    @staticmethod
    def des_s_box4():
        return SBox.from_list([
            [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
            [13, 8, 10, 1, 3, 15, 12, 9, 7, 14, 2, 11, 5, 0, 4, 6],
            [1, 15, 13, 8, 10, 3, 7, 9, 14, 2, 11, 5, 0, 12, 4, 6],
            [10, 9, 11, 0, 8, 7, 15, 1, 3, 14, 5, 13, 12, 2, 4, 6]
        ])
    
    @staticmethod
    def des_s_box5():
        return SBox.from_list([
            [9, 14, 15, 5, 0, 3, 8, 12, 13, 1, 7, 2, 10, 6, 4, 11],
            [8, 6, 3, 14, 13, 4, 7, 11, 10, 2, 9, 15, 12, 0, 5, 1],
            [12, 3, 13, 8, 14, 5, 9, 7, 15, 1, 10, 11, 0, 4, 2, 6],
            [15, 5, 10, 3, 12, 9, 0, 6, 1, 8, 14, 7, 13, 4, 2, 11]
        ])
    
    @staticmethod
    def des_s_box6():
        return SBox.from_list([
            [9, 0, 5, 12, 10, 15, 7, 13, 11, 8, 3, 14, 1, 6, 4, 2],
            [13, 8, 11, 0, 7, 9, 6, 10, 1, 14, 5, 2, 15, 4, 12, 3],
            [10, 5, 8, 6, 11, 9, 0, 3, 1, 15, 4, 13, 12, 7, 14, 2],
            [9, 7, 10, 5, 14, 2, 1, 13, 11, 3, 0, 8, 15, 4, 6, 12]
        ])
    
    @staticmethod
    def des_s_box7():
        return SBox.from_list([
            [9, 12, 15, 10, 8, 7, 2, 3, 1, 14, 5, 11, 13, 0, 6, 4],
            [6, 2, 13, 14, 12, 8, 10, 1, 9, 7, 5, 0, 11, 4, 15, 3],
            [10, 8, 7, 9, 13, 12, 15, 14, 5, 11, 3, 0, 1, 6, 4, 2],
            [15, 6, 12, 3, 10, 1, 4, 7, 8, 14, 9, 13, 11, 5, 2, 0]
        ])
    
    @staticmethod
    def des_s_box8():
        return SBox.from_list([
            [7, 10, 5, 13, 0, 15, 2, 1, 12, 6, 9, 3, 8, 11, 4, 14],
            [15, 10, 6, 0, 1, 14, 13, 9, 11, 8, 12, 5, 7, 3, 2, 4],
            [1, 11, 5, 13, 8, 6, 12, 9, 10, 0, 15, 2, 14, 3, 7, 4],
            [9, 1, 13, 8, 7, 10, 3, 4, 2, 0, 5, 14, 12, 15, 6, 11]
        ])
    

    



In [21]:
# Creamos una SBox personalizada con ntra. propia función
# Analice y explique como está funcionando de forma similar
# esta pequeña caja de substitucion con las Sboxes
# de las Tablas 3.2-3.9  
s_box = SBox(block_size=2, table={
    (0, 0): 5,
    (0, 1): 6,
    (1, 0): 8,
    (1, 1): 7
}, func=lambda x: (x[0], x[1]))

print(s_box('00'))
print(s_box('01'))
print(s_box('10'))
print(s_box('11'))

101
110
1000
111


In [22]:
# Podemos usar el SBox de primera sustitución incorporado para comprimir 
# cadenas binarias de 6 bits -> 4 bits

s_box2 = SBox.des_s_box1()
binary = '000100'  # De la Tabla 3.2 vea que la codificacion debe de dar 13d = 1101b
print(s_box2(binary))

1101


In [23]:
def int_to_bin(number: int, block_size=8) -> str:
    binary = bin(number)[2:] #convierte a binario el 'number' y regresa un 'string' 
    #'0' * (block_size - len(binary))+binary   #si el tamaño de 'number' es < 32 le agrega 
    # a 'binary' los '0's faltantes hasta 32
    return '0' * (block_size - len(binary)) + binary 


##  Intercambiador
Ahora definimos un intercambiador que aceptará una cadena binaria. 
Divide la cadena en 2 partes y devuelve una cadena binaria intercambiada.

In [24]:
class Swapper:
    def __init__(self, block_size=64):
        self.block_size = block_size

        def encrypt(self, tp):
            der=tp[self.block_size//2:]
            izq=tp[:self.block_size//2]
            return der+izq
        
        def decrypt(self, tp):
            der=tp[self.block_size//2:]
            izq=tp[:self.block_size//2]
            return der+izq


 ## AQUI AGREGARÁ SU CODIGO. ANALICE LA ULTIMA PARTE DE ESTE ARCHIVO

## Null Swapper Intercambiador nulo
Null Swapper no es un objeto criptográfico sino más bien un objeto que hemos creado para tener la misma API que el `Swapper` para que podamos conectar el `NoneSwapper` en lugar de la clase `Swapper` cuando deseamos crear un cifrado que no intercambia bloques de bits.

> No sirve para nada en el algoritmo. Simplemente nos proporciona un mecanismo para crear una Ronda Fiestel sin intercambio que veremos más adelante.

In [25]:
class NoneSwapper:
    def encrypt(self, binary: str) -> str:
        return binary

    def decrypt(self, binary: str) -> str:
        return binary

In [26]:
# No cambia el bloque binario durante el cifrado o descifrado.
swapper = NoneSwapper()
swapper.encrypt('1001')

'1001'

In [27]:
swapper.decrypt('1001')

'1001'

## Mezclador (Mixer)

El mezclador toma un bloque binario de longitud fija y lo divide  en 2 partes. 
Luego, la parte derecha se combina con la clave y se le aplica una función 
no reversible en la parte derecha y la clave. Luego se realiza una XOR (^) 
al resultado con la parte izquierda.

Los resultados finales son
```text
l = l ^ f (r, K)
r = r
```

In [28]:
class Mixer:
    def __init__(self, key: int, func=lambda a, b: a % b, block_size=64,
                 initial_permutation=None, final_permutation=None,
                 substitutions: list = None, substitution_block_size=6):
        self.func = func
        self.block_size = block_size
        self.initial_permutation = PBox.identity(block_size // 2) if initial_permutation is None else initial_permutation
        self.final_permutation = PBox.identity(block_size // 2) if final_permutation is None else final_permutation
        self.substitutions = SBox.des_single_round_substitutions() if substitutions is None else substitutions
        self.substitution_block_size = substitution_block_size
        self.key = key

    def encrypt(self, binary: str) -> str:
        ## COLOQUE AQUI TAMBIEN EL CODIGO QUE AGREGO AL METODO SWAPPER
        r = binary[:self.block_size // 2]
        l = binary[self.block_size // 2:]

        #  PBox de expansión
        r1: str = self.initial_permutation.permutate(r)

        # aplicando la función
        r2: str = int_to_bin(self.func(int(r1, base=2), self.key), block_size=self.initial_permutation.out_degree)

        # aplicando las matrices de substitución
        r3: str = ''
        for i in range(len(self.substitutions)):
            block: str = r2[i * self.substitution_block_size: (i + 1) * self.substitution_block_size]
            r3 += self.substitutions[i](block)

        # aplicando la permutación final
        r3: str = self.final_permutation.permutate(r3)

        # aplicando la xor
        l = int_to_bin(int(l, base=2) ^ int(r3, base=2), block_size=self.block_size // 2)
        return l + r

    def decrypt(self, binary:str) -> str:
        return self.encrypt(binary)

    @staticmethod
    def des_mixer(key: int):
        return Mixer(
          key=key,
          initial_permutation=PBox.des_single_round_expansion(),
          final_permutation=PBox.des_single_round_final(),
          func=lambda a, b: a % b
        )

In [29]:
# Creamos un mezclador DES específico. Eso significa que block_size será 64 y que 
# se utilizarán PBox y SBox específicos de DES.
# También usamos la función mod cuando realizamos una operación no invertible 
# sobre r y Key, por lo tanto f = r % Key
mixer = Mixer.des_mixer(key=3)
number = 9876
binary = int_to_bin(number, block_size=64)
print('Texto simple:', binary)

Texto simple: 0000000000000000000000000000000000000000000000000010011010010100


In [30]:
ciphertext = mixer.encrypt(binary)
print('Texto cifrado:', ciphertext)

Texto cifrado: 1111001111001010110111011010000100000000000000000000000000000000


In [31]:
# Descifrado usando el Mezclador (Mixer)
decrypted = mixer.decrypt(ciphertext)
print('Descifrado:', decrypted)

Descifrado: 1111101111001010111100110001010111110011110010101101110110100001


In [32]:
# imprime la salida printing the integer based output
print(int(decrypted, base=2))

18143581324425485729


In [33]:
# Analice este codigo para que pueda implementar dentro de la clase Swapper 
# el método que 
# 1.- cifra (llamelo encrypt) que recibe 64 bits ('binary'), los divide a la mitad
#    e intercambia y los devuelve en 'l' y 'r' (left y right resp.) mediante 'str's
#
# 2.- descifra (llamelo decrypt) que recibe 64 bits ('binary'), los divide a la mitad
#    e intercambia y los devuelve en 'l' y 'r' (left y right resp.) mediante 'str's

block_size = 80
bin1 = binary[0: block_size // 2]
bin2 = binary[block_size // 2:]
print(bin1)
print(bin2)

0000000000000000000000000000000000000000
000000000010011010010100
